In [8]:
import os
os.environ["DIAMBRAROMSPATH"] = os.getcwd() 
os.environ["DIAMBRA_ENVS"] =  " ".join([f"""127.0.0.1:{50051 + i}""" for i in range(1)])
import diambra
import wandb
from wandb.integration.sb3 import WandbCallback
from diambra.arena.stable_baselines3.make_sb3_env import make_sb3_env, EnvironmentSettings
from diambra.arena import EnvironmentSettings, WrappersSettings, RecordingSettings
from stable_baselines3 import DQN
from diambra.arena import SpaceTypes
from diambra.arena import Roles, SpaceTypes, load_settings_flat_dict
import yaml
import json

from stable_baselines3.common.utils import set_random_seed
import time
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.monitor import Monitor

def build_env(params, train : bool = True, render_mode=None, post_wrappers=None, no_vec=False):
    params["settings"]["action_space"] = SpaceTypes.DISCRETE if params["settings"]["action_space"] == "discrete" else SpaceTypes.MULTI_DISCRETE
    settings = load_settings_flat_dict(EnvironmentSettings, params["settings"])
    # always use left side
    settings.role = Roles.P1
    # Wrappers Settings
    wrappers_settings = load_settings_flat_dict(WrappersSettings, params["wrappers_settings"])
    # Create environment
    if no_vec:
        old = os.environ["DIAMBRA_ENVS"]
        os.environ["DIAMBRA_ENVS"] = " ".join([f"""127.0.0.1:{50051 + i}""" for i in range(1)])
    env, _ = make_sb3_env("sfiii3n", settings, wrappers_settings, render_mode=render_mode)
    if no_vec:
        os.environ["DIAMBRA_ENVS"] = old
    return env

def load_config(cfg_file):
    # Read the cfg file
    yaml_file = open(cfg_file)
    params = yaml.load(yaml_file, Loader=yaml.FullLoader)
    print("Config parameters = ", json.dumps(params, sort_keys=True, indent=4))
    yaml_file.close()
    return params

In [9]:
cfgFile = "/home/chenningcong/Desktop/sfiii/agents/stable_baselines3/cfg_files/sfiii3n/sr6_128x4_das_nc_step_ratio_3_gamma_0.99.yaml"
env = build_env(load_config(cfgFile), True, render_mode='human')
obs = env.reset()

INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...


Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.99,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 3
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

INFO:diambra.arena.engine.interface:... done.
INFO:diambra.arena.arena_gym:EnvironmentSettings(game_id='sfiii3n', frame_shape=(128, 128, 1), step_ratio=3, disable_keyboard=True, disable_joystick=True, render_mode='human', splash_screen=True, rank=0, env_address='127.0.0.1:50051', grpc_timeout=600, seed=1733782036, difficulty=6, continue_game=0.0, show_final=False, tower=3, _last_seed=1733782036, pb_model=game_id: "sfiii3n"
frame_shape {
  h: 128
  w: 128
  c: 1
}
step_ratio: 3
n_players: 1
disable_keyboard: true
disable_joystick: true
action_spaces: DISCRETE
episode_settings {
}
, n_players=1, action_space=1, role=2, characters='Ryu', outfits=2, super_art=None, fighting_style=None, ultimate_style=None, speed_mode=None)


In [12]:
for i in obs:
    print(i, obs[i].shape)
obs

action (1, 180)
frame (1, 128, 128, 4)
opp_character (1, 20)
opp_health (1, 1)
opp_side (1,)
own_health (1, 1)
own_side (1,)
stage (1, 1)
timer (1, 1)


OrderedDict([('action',
              array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 0, 0]], dtype=int8)),
             ('frame',
              array([[[[ 43,  43,  43,  43],
                       [ 43,  43,  43,  43],
                       [ 43,  43,  43,  43],
                       ...,
                       [ 48,  48,  4

INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:... done.


Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.99,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 3
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

INFO:diambra.arena.arena_gym:EnvironmentSettings(game_id='sfiii3n', frame_shape=(128, 128, 1), step_ratio=3, disable_keyboard=True, disable_joystick=True, render_mode='human', splash_screen=True, rank=0, env_address='127.0.0.1:50051', grpc_timeout=600, seed=1733781753, difficulty=6, continue_game=0.0, show_final=False, tower=3, _last_seed=1733781753, pb_model=game_id: "sfiii3n"
frame_shape {
  h: 128
  w: 128
  c: 1
}
step_ratio: 3
n_players: 1
disable_keyboard: true
disable_joystick: true
action_spaces: DISCRETE
episode_settings {
}
, n_players=1, action_space=1, role=2, characters='Ryu', outfits=2, super_art=None, fighting_style=None, ultimate_style=None, speed_mode=None)


In [2]:
import torch
import torch.utils
from torch.utils.data.dataloader import DataLoader, default_collate
def collate_fn(x):
    return default_collate([i.observation for i in x]), default_collate([i.action for i in x]), default_collate([i.reward for i in x])

from diambra.arena.stable_baselines3.sb3_utils import linear_schedule
def build_model(cfg_file):
    # PPO settings
    yaml_file = open(cfg_file)
    params = yaml.load(yaml_file, Loader=yaml.FullLoader)
    print("Config parameters = ", json.dumps(params, sort_keys=True, indent=4))
    yaml_file.close()
    ppo_settings = params["ppo_settings"]
    gamma = ppo_settings["gamma"]
    model_checkpoint = ppo_settings["model_checkpoint"]

    learning_rate = linear_schedule(ppo_settings["learning_rate"][0], ppo_settings["learning_rate"][1])
    clip_range = linear_schedule(ppo_settings["clip_range"][0], ppo_settings["clip_range"][1])
    clip_range_vf = clip_range
    batch_size = ppo_settings["batch_size"]
    n_epochs = ppo_settings["n_epochs"]
    n_steps = ppo_settings["n_steps"]
    policy_kwargs = params["policy_kwargs"]
    agent = PPO("MultiInputPolicy", env, verbose=1,
                    gamma=gamma, batch_size=batch_size,
                    n_epochs=n_epochs, n_steps=n_steps,
                    learning_rate=learning_rate, clip_range=clip_range,
                    clip_range_vf=clip_range_vf, policy_kwargs=policy_kwargs, stats_window_size=10)
    return agent


In [3]:
from dataclasses import dataclass
from typing import Any
from tqdm.autonotebook import tqdm
@dataclass
class RolloutSample:
    observation : Any
    action : Any
    reward : Any

def flatten_value(x, t):
    return [{i:x[i][index] for i in x} for index in range(t)]
def flatten_array(x, t):
    return [x[i] for i in range(t)]
def flatten_rollout(observation, action, reward, t):
    return [RolloutSample(obs, a, r) for (obs, a, r) in zip(flatten_value(observation, t), flatten_array(action,t), flatten_array(reward, t))]

def rollout(agent, env, total_step : int, deterministic=False):    
    observation = env.reset()
    cumulative_reward = 0
    result = []
    for i in tqdm(range(0, total_step, env.num_envs)):
        # the shape here is (N, H, W, C)
        action, _state = agent.predict(observation, deterministic=deterministic)
        new_observation, reward, done, info = env.step(action)
        cumulative_reward += reward
        result.extend(flatten_rollout(observation, action, reward, env.num_envs))
        observation = new_observation
    return result

def expert_mark(expert, rollouts):
    return [RolloutSample(r.observation, expert.predict(r.observation, deterministic=False)[0], r.reward) for r in rollouts]

/tmp/ipykernel_335266/1130651163.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
from stable_baselines3.common.evaluation import evaluate_policy
import os
import pickle

from stable_baselines3 import PPO
cfgFile = "/home/chenningcong/Desktop/sfiii/agents/stable_baselines3/cfg_files/sfiii3n/sr6_128x4_das_nc_step_ratio_3_gamma_0.99.yaml"
env = build_env(load_config(cfgFile), True, render_mode='human')

trainedModel = "/home/chenningcong/Desktop/sfiii/agents/stable_baselines3/wandb/latest-run/files/model.zip"
expert = PPO.load(trainedModel)
if os.path.exists("data.pkl"):
    with open('data.pkl', 'rb') as file:
        result = pickle.load(file)
else:
    result = rollout(expert, env, total_step=20000)
    # Open a file in binary write mode
    import pickle
    with open('data.pkl', 'wb') as file:
        # Dump the object to the file
        pickle.dump(result, file)


INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:... done.


Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.99,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 3
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

INFO:diambra.arena.arena_gym:EnvironmentSettings(game_id='sfiii3n', frame_shape=(128, 128, 1), step_ratio=3, disable_keyboard=True, disable_joystick=True, render_mode='human', splash_screen=True, rank=0, env_address='127.0.0.1:50051', grpc_timeout=600, seed=1733591657, difficulty=6, continue_game=0.0, show_final=False, tower=3, _last_seed=1733591657, pb_model=game_id: "sfiii3n"
frame_shape {
  h: 128
  w: 128
  c: 1
}
step_ratio: 3
n_players: 1
disable_keyboard: true
disable_joystick: true
action_spaces: DISCRETE
episode_settings {
}
, n_players=1, action_space=1, role=2, characters='Ryu', outfits=2, super_art=None, fighting_style=None, ultimate_style=None, speed_mode=None)


In [10]:
from tqdm.autonotebook import tqdm
class CallbackHelper:
    def __init__(self, total) -> None:
        self.total = 0
        self.stage = 0
        self.returns = [[] for i in range(10)]
        self.pbar = tqdm(total=total) 
    def evaluate_callback(self, l, g):
        reward = l["reward"]
        d = l["done"]
        obs = l["observations"]
        
        self.total += reward
        cur_stage = int(round(9*obs['stage'].item()))
        if d or int(cur_stage) != self.stage:
            self.returns[self.stage].append(self.total)
            print(self.returns)
            self.total = 0
            if d:
                self.pbar.update(1)
                self.stage = 0
            else:
                self.stage = cur_stage
def evaluate_agent(env, agent, total, render=False):
    env.reset()
    h = CallbackHelper(total=total)
    stats = evaluate_policy(agent, 
                    env,
                    callback = h.evaluate_callback,
                    n_eval_episodes=total, 
                    deterministic=False, render=render)
    return stats, h

In [6]:
new_agent = build_model(cfgFile)
evaluate_agent(env, new_agent, 16, render=True)

Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.99,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 3
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

/home/chenningcong/miniconda3/envs/myenv/lib/python3.11/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 128`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 128
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=128 and n_envs=1)
  warnings.warn(
  0%|          | 0/16 [00:00<?, ?it/s]

[[1.7763974890112877], [], [], [], [], [], [], [], [], []]
[[1.7763974890112877], [1.3788820207118988], [], [], [], [], [], [], [], []]


  6%|▋         | 1/16 [01:20<20:00, 80.03s/it]

[[1.7763974890112877], [1.3788820207118988], [-1.428571529686451], [], [], [], [], [], [], []]
[[1.7763974890112877, 1.6397515684366226], [1.3788820207118988], [-1.428571529686451], [], [], [], [], [], [], []]
[[1.7763974890112877, 1.6397515684366226], [1.3788820207118988, 2.6956521794199944], [-1.428571529686451], [], [], [], [], [], [], []]
[[1.7763974890112877, 1.6397515684366226], [1.3788820207118988, 2.6956521794199944], [-1.428571529686451, 0.44720494002103806], [], [], [], [], [], [], []]


...

 94%|█████████▍| 15/16 [17:16<00:54, 54.27s/it]

[[1.7763974890112877, 1.6397515684366226, 2.3602484613656998, 1.8757763504981995, 0.7204969599843025, 1.5031056329607964, 1.6149068623781204, 0.17391306161880493, 0.9565217718482018, 1.8136645779013634, 0.894410029053688, -0.9440993145108223, 1.3291925191879272, -0.2981366589665413, -0.8819876164197922], [1.3788820207118988, 2.6956521794199944, 1.440993756055832, 0.6583850979804993, -1.391304388642311, -0.062111832201480865, 0.049689412117004395, -0.770186297595501, 1.9999999850988388, -1.2049689590930939, 0.11180123686790466], [-1.428571529686451, 0.44720494002103806, -0.5714286416769028, -1.776397593319416, -2.385093219578266, -1.1304348483681679, 0.16149063408374786], [-2.3478260412812233, -1.5279503092169762], [], [], [], [], [], []]


100%|██████████| 16/16 [17:50<00:00, 48.41s/it]

[[1.7763974890112877, 1.6397515684366226, 2.3602484613656998, 1.8757763504981995, 0.7204969599843025, 1.5031056329607964, 1.6149068623781204, 0.17391306161880493, 0.9565217718482018, 1.8136645779013634, 0.894410029053688, -0.9440993145108223, 1.3291925191879272, -0.2981366589665413, -0.8819876164197922, -0.6956521719694138], [1.3788820207118988, 2.6956521794199944, 1.440993756055832, 0.6583850979804993, -1.391304388642311, -0.062111832201480865, 0.049689412117004395, -0.770186297595501, 1.9999999850988388, -1.2049689590930939, 0.11180123686790466], [-1.428571529686451, 0.44720494002103806, -0.5714286416769028, -1.776397593319416, -2.385093219578266, -1.1304348483681679, 0.16149063408374786], [-2.3478260412812233, -1.5279503092169762], [], [], [], [], [], []]


((0.511646, 1.3278802153670526), <__main__.CallbackHelper at 0x7f0edbbdab50>)

In [11]:
import wandb

In [12]:
def per_level_mean(x):
    if len(x) == 0:
        return -4
    else:
        return np.mean(np.array(x))
import torch.nn as nn
import numpy as np
USE_DEBUG = False
total_step = 100 * len(result)
cur_step = 0
eval_step = 100
if USE_DEBUG:
    eval_n = 1
else:
    eval_n = 16
new_agent = build_model(cfgFile)
loader = DataLoader(result, shuffle=True, batch_size=32, collate_fn=collate_fn)
optimizer = torch.optim.Adam(new_agent.policy.parameters(), lr=2e-4)

wandb.init(project='imitation', name='bc', mode='disabled' if USE_DEBUG else None)
while True:
    cur_loss = 0
    for epoch in range(total_step // len(result)):
        pbar = tqdm(loader)
        for batch in pbar:
            if cur_step % eval_step == 0:
                (e_m, e_std), h = evaluate_agent(env, new_agent, eval_n)
                l = [per_level_mean(i) for i in h.returns]
                log_data = {'eval/mean' : e_m, 'eval/std' : e_std}
                if cur_loss != 0:
                    log_data['loss'] = cur_loss / eval_step 
                #  reset cur_loss
                cur_loss = 0
                for i in range(len(l)):
                    log_data[f"eval/level-{i}"] = l[i]
                print(log_data)
                wandb.log(log_data)
            observation, action, reward = batch
            obs_tensor, vectorized_env = new_agent.policy.obs_to_tensor(observation)
            dist = new_agent.policy.get_distribution(obs_tensor)
            logit = dist.distribution.logits
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logit, action.cuda())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.set_postfix({"loss" :loss.item()})
            cur_loss += loss.item()
            cur_step += 1

Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.99,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 3
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

/home/chenningcong/miniconda3/envs/myenv/lib/python3.11/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 256, but because the `RolloutBuffer` is of size `n_steps * n_envs = 128`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 128
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=128 and n_envs=1)
  warnings.warn(


[[-1.3540372848510742], [], [], [], [], [], [], [], [], []]
[[-1.3540372848510742, 1.4906831681728363], [], [], [], [], [], [], [], [], []]
[[-1.3540372848510742, 1.4906831681728363], [1.4161490425467491], [], [], [], [], [], [], [], []]


[[-1.3540372848510742, 1.4906831681728363], [1.4161490425467491], [-2.447204940021038], [], [], [], [], [], [], []]


[[-1.3540372848510742, 1.4906831681728363, -1.8385092541575432], [1.4161490425467491], [-2.447204940021038], [], [], [], [], [], [], []]
[[-1.3540372848510742, 1.4906831681728363, -1.8385092541575432, 1.652173936367035], [1.4161490425467491], [-2.447204940021038], [], [], [], [], [], [], []]


...

[[2.5093167424201965, 1.2422360330820084, 2.4099378883838654, 2.844720460474491, 2.3602485060691833, 1.900621123611927, 1.614906869828701, -0.6956521794199944, 2.5341614931821823, 2.1366460099816322, 1.0931676551699638], [0.9565216153860092, 1.4782607927918434, 0.9192546382546425, -1.7267081588506699, 1.3416149392724037, -1.0062111020088196, 0.8074533715844154, -1.2919255495071411, 1.3043477982282639, 1.180124171078205], [1.242236040532589, 0.260869562625885, 2.0496894866228104, -0.931677021086216, -1.204968936741352, 0.7453415095806122, -1.1428571864962578], [-0.9813664630055428, -0.993788905441761, -1.4906832426786423], [], [], [], [], [], []]
[[2.5093167424201965, 1.2422360330820084, 2.4099378883838654, 2.844720460474491, 2.3602485060691833, 1.900621123611927, 1.614906869828701, -0.6956521794199944, 2.5341614931821823, 2.1366460099816322, 1.0931676551699638, 1.552795097231865], [0.9565216153860092, 1.4782607927918434, 0.9192546382546425, -1.7267081588506699, 1.3416149392724037, -1.0

[[2.5093167424201965, 1.2422360330820084, 2.4099378883838654, 2.844720460474491, 2.3602485060691833, 1.900621123611927, 1.614906869828701, -0.6956521794199944, 2.5341614931821823, 2.1366460099816322, 1.0931676551699638, 1.552795097231865], [0.9565216153860092, 1.4782607927918434, 0.9192546382546425, -1.7267081588506699, 1.3416149392724037, -1.0062111020088196, 0.8074533715844154, -1.2919255495071411, 1.3043477982282639, 1.180124171078205, 2.5838509276509285], [1.242236040532589, 0.260869562625885, 2.0496894866228104, -0.931677021086216, -1.204968936741352, 0.7453415095806122, -1.1428571864962578, 2.8447205126285553], [-0.9813664630055428, -0.993788905441761, -1.4906832426786423, 1.0062112137675285], [-1.3913043960928917], [], [], [], [], []]
[[2.5093167424201965, 1.2422360330820084, 2.4099378883838654, 2.844720460474491, 2.3602485060691833, 1.900621123611927, 1.614906869828701, -0.6956521794199944, 2.5341614931821823, 2.1366460099816322, 1.0931676551699638, 1.552795097231865, 2.8322981

[[2.5093167424201965, 1.2422360330820084, 2.4099378883838654, 2.844720460474491, 2.3602485060691833, 1.900621123611927, 1.614906869828701, -0.6956521794199944, 2.5341614931821823, 2.1366460099816322, 1.0931676551699638, 1.552795097231865, 2.8322981894016266], [0.9565216153860092, 1.4782607927918434, 0.9192546382546425, -1.7267081588506699, 1.3416149392724037, -1.0062111020088196, 0.8074533715844154, -1.2919255495071411, 1.3043477982282639, 1.180124171078205, 2.5838509276509285, -0.45962733030319214], [1.242236040532589, 0.260869562625885, 2.0496894866228104, -0.931677021086216, -1.204968936741352, 0.7453415095806122, -1.1428571864962578, 2.8447205126285553], [-0.9813664630055428, -0.993788905441761, -1.4906832426786423, 1.0062112137675285], [-1.3913043960928917], [], [], [], [], []]
[[2.5093167424201965, 1.2422360330820084, 2.4099378883838654, 2.844720460474491, 2.3602485060691833, 1.900621123611927, 1.614906869828701, -0.6956521794199944, 2.5341614931821823, 2.1366460099816322, 1.0931

 52%|█████▏    | 325/625 [49:33<45:44,  9.15s/it, loss=2.04]


KeyboardInterrupt: 

In [ ]:
total_step = 100 * len(result)
cur_step = 0
eval_step = 1000
eval_n = 4
new_agent = build_model(cfgFile)
loader = DataLoader(result, shuffle=True, batch_size=32, collate_fn=collate_fn)
optimizer = torch.optim.Adam(new_agent.policy.parameters(), lr=2e-4)

wandb.init(project='imitation', name='bc', mode='disabled')
while True:
    for epoch in range(total_step // len(result)):
        pbar = tqdm(loader)
        for i in pbar:
            observation, action, reward = i
            obs_tensor, vectorized_env = new_agent.policy.obs_to_tensor(observation)
            dist = new_agent.policy.get_distribution(obs_tensor)
            logit = dist.distribution.logits
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logit, action.cuda())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.set_postfix({"loss" :loss.item()})
            cur_step += 1
            
            if (cur_step + 1) % eval_step == 0:
                (e_m, e_std), h = evaluate_agent(env, new_agent, eval_n)
                log_data = {'loss' : loss.item(), 'eval/mean' : e_m, 'eval/std' : e_std, }
                print(log_data)
                wandb.log(log_data)


In [8]:
import torch.nn as nn
# dagger
new_agent = build_model(cfgFile)
optimizer = torch.optim.Adam(new_agent.policy.parameters(), lr=1e-4)
rollout_buffer = result.copy()
for epoch in range(100):
    loader = DataLoader(rollout_buffer, shuffle=True, batch_size=32, collate_fn=collate_fn)
    pbar = tqdm(loader)
    for i in pbar:
        observation, action, reward = i
        obs_tensor, vectorized_env = new_agent.policy.obs_to_tensor(observation)
        dist = new_agent.policy.get_distribution(obs_tensor)
        logit = dist.distribution.logits
        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logit, action.cuda())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"loss" :loss.item()})
    # after training, we rollout the agent to collect OOD data
    new_data = rollout(new_agent, env, total_step=2000)
    # The expert marks the data
    new_data = expert_mark(expert, new_data)
    # append new data
    rollout_buffer.extend(new_data)
    if epoch > 0 and epoch % 10 == 0:
        print(evaluate_policy(new_agent, env, n_eval_episodes=16, deterministic=False))


Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.99,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 3
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

  0%|          | 0/688 [00:00<?, ?it/s]/home/chenningcong/miniconda3/envs/myenv/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py:227: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.as_tensor(batch)
100%|██████████| 500/500 [00:06<00:00, 80.61it/s]


(3.2476708125, 2.254294600518592)


100%|██████████| 500/500 [00:06<00:00, 73.54it/s]


(2.3315217500000003, 1.8827598234666543)


100%|██████████| 500/500 [00:06<00:00, 75.36it/s]


(2.7880436250000002, 1.6647524899165895)


100%|█████████▉| 499/500 [11:17<01:04, 64.81s/it]

NameError: name 'render_env' is not defined

: 

In [14]:
def test_agent(env, agent, seed=42, use_sample=False):
    # Environment reset
    observation = env.reset()
    # Agent-Environment interaction loop
    total_reward = 0
    while True:
        env.render()
        # Action random sampling
        if use_sample:
            actions = env.action_space.sample()
        else:
            actions, _state = agent.predict(observation, deterministic=False)
        # Environment stepping
        print(actions)
        observation, reward, done, info = env.step(actions)
        total_reward += reward[0]
        if reward[0] != 0:
            print(reward[0])
        # Episode end (Done condition) check
        if done:
            observation = env.reset()
            break

    # # Environment shutdown
    # env.close()

    # Return success
    return total_reward
render_env = build_env(load_config(cfgFile), True, render_mode="human", no_vec=True)
new_agent = build_model(cfgFile)
test_agent(render_env, agent=expert,use_sample=False)

INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:... done.


Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.99,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 3
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

INFO:diambra.arena.arena_gym:EnvironmentSettings(game_id='sfiii3n', frame_shape=(128, 128, 1), step_ratio=3, disable_keyboard=True, disable_joystick=True, render_mode='human', splash_screen=True, rank=0, env_address='127.0.0.1:50051', grpc_timeout=600, seed=1732658889, difficulty=6, continue_game=0.0, show_final=False, tower=3, _last_seed=1732658889, pb_model=game_id: "sfiii3n"
frame_shape {
  h: 128
  w: 128
  c: 1
}
step_ratio: 3
n_players: 1
disable_keyboard: true
disable_joystick: true
action_spaces: DISCRETE
episode_settings {
}
, n_players=1, action_space=1, role=None, characters='Ryu', outfits=2, super_art=None, fighting_style=None, ultimate_style=None, speed_mode=None)


Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.99,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 3
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

1.7888196483254433

In [15]:
render_env = build_env(load_config(cfgFile), True, render_mode=None)
evaluate_policy(expert, env, n_eval_episodes=16, deterministic=False)

Config parameters =  {
    "folders": {
        "model_name": "sr6_128x4_das_nc",
        "parent_dir": "./results/"
    },
    "policy_kwargs": {
        "net_arch": [
            64,
            64
        ]
    },
    "ppo_settings": {
        "autosave_freq": 256,
        "batch_size": 256,
        "clip_range": [
            0.15,
            0.025
        ],
        "gamma": 0.99,
        "learning_rate": [
            0.00025,
            2.5e-06
        ],
        "model_checkpoint": "0",
        "n_epochs": 4,
        "n_steps": 128,
        "time_steps": 10000000
    },
    "settings": {
        "action_space": "discrete",
        "characters": "Ryu",
        "continue_game": 0.0,
        "difficulty": 6,
        "frame_shape": [
            128,
            128,
            1
        ],
        "game_id": "sfiii3n",
        "outfits": 2,
        "step_ratio": 3
    },
    "wrappers_settings": {
        "add_last_action": true,
        "dilation": 1,
        "exclude_image_sc

INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:... done.
INFO:diambra.arena.engine.interface:... done.
INFO:diambra.arena.engine.interface:... done.
INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:... done.
INFO:diambra.arena.arena_gym:EnvironmentSettings(game_id='sfiii3n', frame_shape=(128, 128, 1), step_ratio=3, disable_keyboard=True, disable_joystick=True, render_mode=None, splash_screen=True, rank=1, env_address='127.0.0.1:50052', grpc_timeout=600, seed=1732659101, difficulty=6, continue_game=0.0, show_final=False, tower=3, _last_seed=1732659101, pb_model=game_id: "sfiii3n"
frame_shape {
  h: 128
  w: 128
  c: 1
}
step_rati

(2.5605590000000005, 1.889581989302462)